### 카테고리별 지원 파싱 (장애인지원)
장애인지원 (238~293페이지) 파싱

In [18]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

load_dotenv()

True

In [19]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [20]:
def extract_page_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [21]:
def parse_disability_support(text, page_num):
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """장애인지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "장애인 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "장애인연금", "장애인활동지원", "발달장애인 부모상담지원")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["장애등급", "월지급액", "비고"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["중증", "43만원", "기초급여+부가급여"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 장애인지원제도명만
- 대상: 장애유형, 장애등급, 소득기준 등 구체적 조건
- 내용: 지원금액, 지원기간, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """장애인지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "장애인 지원" 제외
- 실제 장애인지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    chain = prompt | llm | parser
    
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "장애인지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)
            else:
                return None

In [15]:
# 실행
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(238, 294))  # 장애인지원 섹션 (238~293페이지)
disability_programs = []

for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    page_text = extract_page_text(pdf_path, page_num)
    program = parse_disability_support(page_text, page_num)
    
    if program:
        disability_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f" {program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 238 파싱 중... (1/56)
 장애인 지원 추출 완료 (표 0개)
페이지 239 파싱 중... (2/56)
 장애인 서비스지원 종합조사 신청 추출 완료 (표 0개)
페이지 240 파싱 중... (3/56)
 장애아동수당 추출 완료 (표 1개)
페이지 241 파싱 중... (4/56)
 장애수당 추출 완료 (표 1개)
페이지 242 파싱 중... (5/56)
 장애인연금 추출 완료 (표 1개)
페이지 243 파싱 중... (6/56)
 여성장애인 출산비용 지원 추출 완료 (표 0개)
페이지 244 파싱 중... (7/56)
 장애아 보육료 지원 추출 완료 (표 1개)
페이지 245 파싱 중... (8/56)
 발달장애인 가족휴식 지원 추출 완료 (표 0개)
페이지 246 파싱 중... (9/56)
 발달장애인 부모 상담 지원 추출 완료 (표 0개)
페이지 247 파싱 중... (10/56)
 장애아 방과후 보육료 지원 추출 완료 (표 0개)
페이지 248 파싱 중... (11/56)
 특수교육대상자 치료지원 서비스 추출 완료 (표 0개)
페이지 249 파싱 중... (12/56)
 국립특수학교 및 국립부설학교 특수학급 지원 추출 완료 (표 0개)
페이지 250 파싱 중... (13/56)
 장애인 정보화교육 추출 완료 (표 1개)
페이지 251 파싱 중... (14/56)
 장애인 일자리 지원 추출 완료 (표 1개)
페이지 252 파싱 중... (15/56)
 중증장애인 직업재활 지원 추출 완료 (표 0개)
페이지 253 파싱 중... (16/56)
 장애인직업재활시설 이용 추출 완료 (표 1개)
페이지 254 파싱 중... (17/56)
 중증장애인생산품 우선구매 추출 완료 (표 0개)
페이지 255 파싱 중... (18/56)
 장애인 취업성공패키지 추출 완료 (표 1개)
페이지 256 파싱 중... (19/56)
 중증장애인 근로지원인 추출 완료 (표 0개)
페이지 257 파싱 중... (20/56)
 장애인 신규 고용장려금 지원 추출 완료 (

In [22]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/장애인지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(disability_programs, f, ensure_ascii=False, indent=2)

print(f"\n 파싱 완료: {len(disability_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


 파싱 완료: 55개 프로그램
저장 위치: ../../data/outputs/장애인지원_2025.json


In [24]:

for program in disability_programs:
    table_count = len(program.get('tables', []))
    print(f"\n{program['title']} (페이지 {program['page']}, 표 {table_count}개)")
    print(f"대상: {program['target'][:80]}...")
    print(f"내용: {program['content'][:80]}...")
    if table_count > 0:
        print(f"표: {program['tables'][0]['columns'][:3]}...")


장애인 지원 (페이지 238, 표 0개)
대상: 장애로 인해 생활이 곤란한 중증장애인...
내용: 매월 일정 금액을 지원하여 안정적으로 생활할 수 있도록 지원합니다....

장애인 서비스지원 종합조사 신청 (페이지 239, 표 0개)
대상: 등록 장애인...
내용: 서비스지원 종합조사를 통해 대상자를 선정하고 필요한 서비스를 제공합니다. 대상은 ① 일상생활지원 분야(활동지원, 거주시설, 보조기기, 주간활동,...

장애아동수당 (페이지 240, 표 1개)
대상: 18세 미만의 등록 장애아동 중 기초생활수급자 또는 차상위계층...
내용: 18세 미만의 등록 장애아동 중 기초생활수급자 또는 차상위계층을 대상으로 하며, 소득 수준 및 주거형태에 따라 매월 장애아동수당을 지급합니다. ...
표: ['구분', '중증장애아동수당', '경증장애아동수당']...

장애수당 (페이지 241, 표 1개)
대상: 18세 이상의 장애인연금법상 중증장애인에 해당하지 않는 등록장애인 중 기초생활수급자 또는 차상위계층...
내용: 18세 이상의 중증장애인에 해당하지 않는 등록장애인 중 기초생활수급자 또는 차상위계층에게 소득 수준 및 주거형태에 따라 매월 장애수당을 지급합니...
표: ['구분', '장애수당', '장애수당']...

장애인연금 (페이지 242, 표 1개)
대상: 18세 이상의 장애인연금법상 중증장애인* 중 본인과 배우자의 소득인정액이 선정기준액 이하에 해당하는 자...
내용: 18세 이상의 중증장애인에게 장애인연금(기초급여, 부가급여)를 지원합니다. 월지급액은 43만 2,510원부터 시작하여 차등 지급됩니다. 65세 ...
표: ['구분', '18세 ~ 64세', '65세 이상**']...

여성장애인 출산비용 지원 (페이지 243, 표 0개)
대상: 출산(임신기간 4개월 이상 유산, 사산 포함)한 여성 장애인...
내용: 출산(유산, 사산 포함) 태아 1인 기준 120만 원 지급. 해산급여, 첫만남이용권과 중복지원 가능. 신청에 필요한 서류는 신청자 신분증(대리 ...